In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

import sys
import os

import multiprocessing

In [ ]:
def transform_data(file_pre,filename,overwrite=False):
    # Create the file, if it does not exist yet
    if os.path.exists(filename):
        if (overwrite == False):
            print('File does already exist. Do nothing')
            return
        else:
            print('File does already exist. Overwrite.')
            
    file_index  = 0
    f_read      = h5py.File(file_pre + str(file_index).zfill(4) + ".h5",'r')
    file_length = len(f_read['circuit_0'].keys())
    file_loc    = [0]*15

    def get_next_data(circuit_id,f_read,file_index,file_length,file_loc):
        data    = f_read['circuit_'+str(circuit_id)+'/event_'+str(file_loc[circuit_id])]
        data    = {'pattern'  : np.array(data['pattern_0']),
                   'spikes_1' : np.array(data['spikes_1']),
                   'spikes_2' : np.array(data['spikes_2']),
                   'spikes_3' : np.array(data['spikes_3']),
                   'spikes_4' : np.array(data['spikes_4'])}
        file_loc[circuit_id] = file_loc[circuit_id] + 1

        if circuit_id == 14 and file_loc[circuit_id] == file_length:
            f_read.close()
            file_index = file_index + 1
               
            try:
                f_read      = h5py.File(file_pre + str(file_index).zfill(4) + ".h5",'r')
                file_length = file_length + len(f_read['circuit_0'].keys())
            except:
                print('\n\rAll files read out')
                return data,  True,   None,    None,        None,         None

        return         data,  False,  f_read,  file_index,  file_length,  file_loc
    
    unit_length  = 512

    raw_patterns = []
    for i in range(15):
        raw_patterns.append([]) # stim_ID, pattern_ID
        raw_patterns[i].append([])

    raw_spikes   = []
    for i in range(15):
        raw_spikes.append([]) # stim_ID, elec_ID, time_after_stim
        raw_spikes[i].append([])
        
    running_count = 0
    flag = True
    while flag:
        print(running_count,end='\r')
        sys.stdout.flush()

        # Get next data point
        for i in range(15):        
            data,done,f_read,file_index,file_length,file_loc = get_next_data(i,f_read,file_index,file_length,file_loc)

            flag = not done

            pattern = data['pattern']

            if len(raw_patterns[i][-1]) > unit_length:
                raw_patterns[i].append([])
            raw_patterns[i][-1].append([running_count,int(pattern)])

            for j in range(4):
                list_spikes_j = data['spikes_'+str(j+1)].tolist()
                for element in list_spikes_j:
                    if len(raw_spikes[i][-1]) > unit_length:
                        raw_spikes[i].append([])
                    raw_spikes[i][-1].append([running_count,j,element])
        running_count += 1
        
    patterns = []
    for i in range(15):
        patterns.append([np.array(raw_patterns[i][j]) for j in range(len(raw_patterns[i]))])
    patterns = [np.concatenate(patterns[i],0) for i in range(15)]
    
    spikes = []
    for i in range(15):
        spikes.append([np.array(raw_spikes[i][j]) for j in range(len(raw_spikes[i]))])
    spikes = [np.concatenate(spikes[i],0) for i in range(15)]
    
    f = h5py.File(filename,'w')
    try:
        for i in range(15):
            data_group = f.create_group("Circuit_" + str(i))
            data_group.create_dataset("Patterns", data=patterns[i])
            data_group.create_dataset("Spikes",   data=spikes[i])
        f.close()
    except:
        print('')
        print('!!!')
        print('Failed to save the data properly.')
        try:
            f.close()
        except:
            print('Could not close the file.')
        print('!!!')
        print('')

In [ ]:
file_pre  = "filename_of_data_before_transformation_"  # Should end with an underscore and not contain the 4 digit number or file ending (h5)
file_post = "filename_of_data_after_transformation.h5" # Should contain the file ending

In [ ]:
transform_data(file_pre,file_post,overwrite=False)     # Overwrites an old file, if overwrite=True